In [13]:
import logging
import pandas as pd
import os
from typing import Any
from algoshort.regime_fc import RegimeFC
from algoshort.returns import ReturnsCalculator
from algoshort.stop_loss import StopLossCalculator   # your stop-loss module
# from algoshort.position_sizing import PositionSizing  # your position sizing module
from algoshort.utils import load_config
# from algoshort.optimizer import get_equity, StrategyOptimizer
import numpy as np
from algoshort.yfinance_handler import YFinanceDataHandler
from algoshort.ohlcprocessor import OHLCProcessor
from datetime import date
from algoshort.wrappers import generate_signals



In [14]:
# Most common — set once at application start
import logging

logging.basicConfig(
    level=logging.WARNING,              # or DEBUG / WARNING / ERROR
    format="%(asctime)s [%(levelname)7s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [15]:
handler = YFinanceDataHandler(cache_dir="./cache", enable_logging=False)
handler.download_data(['A2A.MI', 'FTSEMIB.MI'], start='2016-01-01', end=date.today(), use_cache=True)
df = handler.get_ohlc_data('A2A.MI')
df['fx'] = 1

bmk = handler.get_ohlc_data('FTSEMIB.MI')

In [16]:
from algoshort.ohlcprocessor import OHLCProcessor
processor = OHLCProcessor()
df = processor.calculate_relative_prices(
    stock_data= df,
    benchmark_data= bmk
    )

2026-02-05 22:40:54 [WARNING] algoshort.ohlcprocessor.OHLCProcessor: Missing benchmark data for 1 rows (0.0%). Values will be forward-filled. This may indicate misaligned date ranges.


In [17]:
tt_search_space = {
    'fast': [20],
    'slow': [50]
}

bo_search_space = [100]

ma_search_space = {
    'short_ma': [50],
    'medium_ma': [100],
    'long_ma': [150]
}


df, signal_columns = generate_signals(
    df=df,
    tt_search_space=tt_search_space,
    bo_search_space=bo_search_space,
    ma_search_space=ma_search_space,
)


In [18]:
signal_columns = [x for x in signal_columns if x != "rrg"]

In [19]:
from algoshort.combiner import SignalGridSearch
searcher = SignalGridSearch(
    df=df,
    available_signals=signal_columns,
    direction_col='rrg'
)

# Run with default settings (all cores, multiprocessing)
results = searcher.run_grid_search_parallel(
    allow_flips=True,
    require_regime_alignment=True
)

df = searcher.df


✓ Validation passed: All 10 signals found in dataframe

GRID GENERATION
Direction column: rrg
Available signals for entry/exit: 10
Signals: ['rbo_100', 'rbo_50', 'rbo_20', 'rtt_5020', 'rsma_50100', 'rsma_100150', 'rsma_50100150', 'rema_50100', 'rema_100150', 'rema_50100150']

Total combinations to test: 100
  = 10 entry signals × 10 exit signals

RUNNING PARALLEL GRID SEARCH
Direction column: rrg
Allow flips: True
Require regime alignment: True
Backend: multiprocessing
Parallel jobs: 22 cores

Processing 100 combinations in parallel...


Testing combinations: 100%|██████████| 100/100 [00:06<00:00, 15.84it/s]



Adding combined signal columns to main dataframe...

GRID SEARCH COMPLETE
Successful combinations: 100
Failed combinations: 0


In [20]:
from algoshort.returns import ReturnsCalculator

calc = ReturnsCalculator(
    df,
    open_col="open",      # adjust once here
    high_col="high",
    low_col="low",
    close_col="close",
    relative_prefix="r"  # or "rel_", "adj_", etc.
)

# Use relative OHLC (r_Open, r_High, ...)
df = calc.get_returns_multiple(df, signals=signal_columns, relative=True)

In [21]:
from algoshort.stop_loss import StopLossCalculator
calc = StopLossCalculator(df)

for signal in signal_columns:
    df = calc.atr_stop_loss(
        signal=signal,
        window=14,
        multiplier=2
    )
    calc.data = df


In [22]:
from algoshort.position_sizing import PositionSizing
from algoshort.position_sizing import run_position_sizing_parallel

sizer = PositionSizing(
    tolerance=-0.1,
    mn=-0.0025,
    mx=-0.05,
    equal_weight=0.05,
    avg=0.03,
    lot=1
)

# signals = signal_columns

df = run_position_sizing_parallel(
    sizer=sizer,
    df=df,
    signals=signal_columns,
    # Customize if your naming uses different suffixes:
    chg_suffix="_chg1D_fx",
    sl_suffix="_stop_loss",
    close_col='close',
    n_jobs=-1,
    verbose=5   # progress output
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    6.8s remaining:   16.1s
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    6.9s remaining:    4.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    7.1s finished


In [24]:
df.filter(like='equity').describe()

,rbo_100_equity_equal,rbo_100_equity_constant,rbo_100_equity_concave,rbo_100_equity_convex,rbo_50_equity_equal,rbo_50_equity_constant,rbo_50_equity_concave,rbo_50_equity_convex,rbo_20_equity_equal,rbo_20_equity_constant,...,rema_50100_equity_concave,rema_50100_equity_convex,rema_100150_equity_equal,rema_100150_equity_constant,rema_100150_equity_concave,rema_100150_equity_convex,rema_50100150_equity_equal,rema_50100150_equity_constant,rema_50100150_equity_concave,rema_50100150_equity_convex
count,2547.000000,2.0,2.0,2.0,2547.000000,2.0,2.0,2.0,2547.000000,2.0,...,2.0,2.0,2547.000000,2.0,2.0,2.0,2547.000000,2.0,2.0,2.0
mean,98871.974143,100000.0,100000.0,100000.0,100492.618438,100000.0,100000.0,100000.0,101412.507029,100000.0,...,100000.0,100000.0,95197.401325,100000.0,100000.0,100000.0,101470.306398,100000.0,100000.0,100000.0
std,654.700262,0.0,0.0,0.0,690.732012,0.0,0.0,0.0,881.181045,0.0,...,0.0,0.0,2331.236383,0.0,0.0,0.0,1244.472534,0.0,0.0,0.0
min,97016.419800,100000.0,100000.0,100000.0,98761.682700,100000.0,100000.0,100000.0,99844.816700,100000.0,...,100000.0,100000.0,91271.752400,100000.0,100000.0,100000.0,98671.567400,100000.0,100000.0,100000.0
25%,98500.930350,100000.0,100000.0,100000.0,100021.474400,100000.0,100000.0,100000.0,100626.940100,100000.0,...,100000.0,100000.0,93274.330000,100000.0,100000.0,100000.0,100109.268950,100000.0,100000.0,100000.0
50%,98759.450700,100000.0,100000.0,100000.0,100441.433100,100000.0,100000.0,100000.0,101254.627500,100000.0,...,100000.0,100000.0,94837.508100,100000.0,100000.0,100000.0,101841.041200,100000.0,100000.0,100000.0
75%,99116.325050,100000.0,100000.0,100000.0,101046.831100,100000.0,100000.0,100000.0,102255.349400,100000.0,...,100000.0,100000.0,96897.855350,100000.0,100000.0,100000.0,102523.115450,100000.0,100000.0,100000.0
max,100855.504000,100000.0,100000.0,100000.0,101886.838500,100000.0,100000.0,100000.0,103219.036500,100000.0,...,100000.0,100000.0,100000.000000,100000.0,100000.0,100000.0,103228.342900,100000.0,100000.0,100000.0


In [11]:
# select date + all equity columns
df_sub = df[['date']].join(df.filter(regex='equity')).tail(1)

# pivot to long format
df_long = df_sub.melt(
    id_vars='date',
    var_name='equity_type',
    value_name='equity_value'
)

df_long["combination_name"] = df_long["equity_type"].str.replace(r"_equity_equal$", "", regex=True)
df_long = df_long.merge(results[['combination_name', 'total_trades']], on='combination_name').sort_values("equity_value", ascending=True) .reset_index(drop=True)
df_long

,date,equity_type,equity_value,combination_name,total_trades
